# Developing behavioral data preprocessing pipeline for OPM MEG analysis


## Preprocessing Steps incuded:
- Loading, syncing, and alligning data 
- Categorizing ACC and EMG data into rest vs muscular activity vs movement
-


### 0. Importing

In [ ]:
# general packages
import json
import os
import importlib
import sys
import numpy as np
import pandas as pd
import pyxdf
from itertools import compress
import matplotlib.pyplot as plt
import datetime as dt

import mne

# ephys packages
# from mne.filter import filter_data, notch_filter

In [ ]:
def add_repo_dir():
    """adds local repo directory to sys to allow importing from repo"""

    wd = os.getcwd()

    COUNTER = 20  #  to prevent eternal while loop

    while not wd.endswith('lid_opm'):
        wd = os.path.dirname(wd)
        COUNTER -= 1

        if COUNTER == 0:
            raise ValueError('repo dir not found!')

    print(f'add repo directory to sys: {wd} ')

    sys.path.append(wd)

    return

In [ ]:
# add custom functions

add_repo_dir()

import utils.load_utils as load_utils
from source_raw_conversion import load_source_opm as source_opm


## 1. Load behavioral source data

Define:
- subject
- task
- configuration version


In [ ]:
CONFIG_VERSION = "v1"

SUB = '03'  # 


# load settings
sub_config = load_utils.load_subject_config(subject_id=SUB,)
preproc_config = load_utils.load_preproc_config(version=CONFIG_VERSION,)
sub_meta_info = load_utils.get_sub_rec_metainfo(config_sub=sub_config)




In [ ]:
sub_meta_info

In [ ]:
import source_raw_conversion.load_lsl as loadlsl
import source_raw_conversion.time_syncing as sync
import source_raw_conversion.load_source_opm as sourceopm

import signal_processing.preprocessing as preproc
import plotting.processing_checks as proc_plotting

manual dev of source lsl to raw flow

In [ ]:
importlib.reload(proc_plotting)
importlib.reload(preproc)
importlib.reload(loadlsl)
importlib.reload(sync)

TASK_sel = 'task'
ACQ_sel = 'dopa45'

for REC in sub_meta_info['rec_name']:
    print(REC)

    try:
        TASK, ACQ = REC.split('_')
    except:
        print(f'\n##### WARNING: {REC} skipped\n')
        continue
    
    if TASK == TASK_sel and ACQ == ACQ_sel:

        if recRaw.task == 'rest': continue 

        recRaw = preproc.rawData_singleRec(SUB, TASK, ACQ, ZSCORE_ACC=True, ZSCORE_EMG=True)


        ### plot not working anymore bcs of missing "task_epochs" attribute
        ### original plot seemed correct from timings; check how timings were extracted then
        proc_plotting.plot_emgacc_check_for_tasks(recRaw, SAVE=False, SHOW=True,
                                                  )
        
        # include sample sizes in plt



In [ ]:
acc = recRaw.ACC.copy()
aux_epochs = recRaw.aux_task_epochs.copy()

In [ ]:
recRaw.aux_chnames

In [ ]:
acc.times

In [ ]:
recRaw.auxtimes

i_test = recRaw.aux_task_epochs['go_left'][0][0]

recRaw.auxtimes[i_test]

In [ ]:
acc.times[aux_epochs['go_left'][0][0]]

In [ ]:
chs = recRaw.ACC.info['ch_names']

# recRaw.ACC.pick('acc_svm_leftfoot').get_data().ravel().shape

chs

In [ ]:
importlib.reload(preproc)
importlib.reload(loadlsl)
importlib.reload(sync)
importlib.reload(source_opm)

TASK = 'task'
ACQ = 'dopa45'


recRaw = preproc.rawData_singleRec(
    SUB, TASK, ACQ, INCL_OPM=True,
    OPM_PREPROC={
        'resample': True, 'bandpass': True,
        'notch': True, 'hfc': False
    },
    ZSCORE_ACC=True,
    ZSCORE_EMG=True,
)


check hfc projections, for now too little sensors probably, therefore no conversion of SVD math

In [ ]:
### HFC CHECK
# hfc_projs = recRaw.OPM_Z.info['projs']

# for p in hfc_projs:
#     # print('Name:', p['desc'], 'Active:', p.get('active', False))
#     cols = p.get('data', {}).get('col_names', None)
#     # print('  cols:', cols)


# # print(recRaw.OPM_Z.ch_names)
# # print(recRaw.OPM_Z.info['bads'])

# print(hfc_projs[0]['data']['col_names']) 
# print(hfc_projs[0]['data']['data'])          # should be your MEG channel names
# print(np.linalg.norm(hfc_projs[0]['data']['data']))  # should NOT be 0

# # check sensor geometry, should not be close to 0
# pos = np.array([ch['loc'][:3] for ch in recRaw.OPM_Z.info['chs']])
# print("Sensor bounds (min, max) in meters:\n", pos.min(axis=0), pos.max(axis=0))

Check orientation of first 3 and last 3 orientation normvector rotations

In [ ]:
# run for 3d turning plot
%matplotlib notebook

### orientation solving

chnames = recRaw.OPM_Z.ch_names

# chname = '15Z'
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for chname in chnames:
    ch = recRaw.OPM_Z.info['chs'][chnames.index(chname)]
    # print(ch['loc'])
    pos = ch['loc'][:3]            # channel position in meters
    ori_z = ch['loc'][3:6]         # first orientation vector
    ori_y = ch['loc'][6:9]         # second orientation vector (if available)

    # Plot sensor location
    ax.scatter(*pos, c='k', s=20,)

    # Plot orientation vectors (scaled for visibility)
    scale = .005
    ax.quiver(*pos, *ori_z, length=scale, color='r',)
    ax.quiver(*pos, *ori_y, length=scale, color='b',)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
ax.set_title(f"Orientation vectors ")
plt.show()

In [ ]:
# set interactive plotting back
%matplotlib inline

## Epoching based on behavioral task


- check (ICA) cleaning before epoching
- TODO select on event type, test topograms per event type

In [ ]:
recRaw.OPM_Z.info

In [ ]:
def get_epochs(acqClass, TMIN=-1, TMAX=3):



    emg_epochs = mne.Epochs(
        raw=acqClass.EMG, events=acqClass.event_arr,
        event_id=acqClass.event_codes,
        tmin=TMIN, tmax=TMAX, baseline=None, preload=True,
        reject=None,
    )
    acc_epochs = mne.Epochs(
        raw=acqClass.ACC, events=acqClass.event_arr,
        event_id=acqClass.event_codes,
        tmin=TMIN, tmax=TMAX, baseline=None, preload=True,
        reject=None,
    )
    opm_epochs = mne.Epochs(
        raw=acqClass.OPM_Z, events=acqClass.event_arr,
        event_id=acqClass.event_codes,
        tmin=TMIN, tmax=TMAX, baseline=None, preload=True,
        reject=None,
    )

    return opm_epochs, emg_epochs, acc_epochs


In [ ]:
sub_meta_info

In [ ]:
importlib.reload(preproc)
importlib.reload(loadlsl)
importlib.reload(sync)
importlib.reload(source_opm)

TASK = 'task'
ACQ = 'dopa60'


recRaw = preproc.rawData_singleRec(
    SUB, TASK, ACQ, INCL_OPM=True,
    OPM_PREPROC={
        'resample': True, 'bandpass': True,
        'notch': True, 'hfc': False
    },
    ZSCORE_ACC=False,
    ZSCORE_EMG=False,
)

TMIN, TMAX = -1, 3

opm_epochs, emg_epochs, acc_epochs = get_epochs(acqClass=recRaw,)

In [ ]:
GO_STIM = 'abort'
TASK_SIDE = 'left'
GOTASK = f'{GO_STIM}_{TASK_SIDE}'
if GO_STIM == 'rest': GOTASK = GO_STIM

figpath = os.path.join(load_utils.get_onedrive_path('figures'),
                        'explore')
fname = f'emgaccCheck_sub{recRaw.sub}_{recRaw.task}_{recRaw.acq}_{GOTASK}'


sfreq = emg_epochs.info['sfreq']

fig, axes = plt.subplots(2, 2, figsize=(9, 9),
                         sharey='row', sharex='col',)
fsize = 14


# select matching channels
meg_ch_match = np.array([TASK_SIDE in ch for ch in emg_epochs.info['ch_names']])
acc_ch_match = np.array([TASK_SIDE in ch for ch in acc_epochs.info['ch_names']])


### plot matching sides
# emg
emg3d = emg_epochs[GOTASK].get_data()[:, meg_ch_match, :]
axes[0, 0].plot(np.mean(emg3d, axis=0).T, alpha=.3,
                label=np.array(emg_epochs.info['ch_names'])[meg_ch_match],
                )
# acc
acc3d = acc_epochs[GOTASK].get_data()[:, acc_ch_match, :]
axes[1, 0].plot(np.mean(acc3d, axis=0).T, alpha=.3,
                label=np.array(acc_epochs.info['ch_names'])[acc_ch_match],
                )

axes[0, 0].set_title('Matching sides to task', size=fsize)
axes[0, 0].set_ylabel('EMG-envelop (V)', size=fsize)
axes[1, 0].set_xlabel('Time vs TASK-onset (sec)', size=fsize)



### plot non-matching sides
# select NON-matching channels
print(f'include for NONMATCH: {np.array(emg_epochs.info["ch_names"])[~meg_ch_match]}')
# emg
emg3d = emg_epochs[GOTASK].get_data()[:, ~meg_ch_match, :]
axes[0, 1].plot(np.mean(emg3d, axis=0).T, alpha=.3,
                label=np.array(emg_epochs.info['ch_names'])[~meg_ch_match],
)
if recRaw.ZSCORE_EMG:
    axes[0, 0].set_ylim(-1, 3)
    axes[0, 1].set_ylim(-1, 3)
else:
    axes[0, 0].set_ylim(0, 3.5e-5)
    axes[0, 1].set_ylim(0, 3.5e-5)


# acc
acc3d = acc_epochs[GOTASK].get_data()[:, ~acc_ch_match, :]
axes[1, 1].plot(np.mean(acc3d, axis=0).T, alpha=.3,
                label=np.array(acc_epochs.info['ch_names'])[~acc_ch_match],
)
if recRaw.ZSCORE_ACC:
    axes[1, 1].set_ylim(-1, 3)
    axes[1, 0].set_ylim(-1, 3)
else:
    axes[1, 1].set_ylim(0, .01)
    axes[1, 0].set_ylim(0, .01)


axes[0, 1].set_title('Non-matching sides to task', size=fsize)
axes[1, 0].set_ylabel('ACC-magn. vector (g)', size=fsize)
axes[1, 1].set_xlabel('Time vs trial-onset (sec)', size=fsize)



for ax in axes.ravel():
    ax.tick_params(size=fsize, labelsize=fsize,)
    xtlabels = np.arange(TMIN, TMAX+.1, 1)
    ax.set_xticks([sfreq] * np.arange(len(xtlabels)),)
    ax.set_xticklabels(xtlabels)

    ax.legend()

    ax.axvline(-1*TMIN*sfreq, ymin=0, ymax=1, color='green', lw=3, alpha=.3,)

if GO_STIM == 'rest': TASK_SIDE = f'"{TASK_SIDE}"'

plt.suptitle(f'Task selection: {GO_STIM}: {TASK_SIDE}, {recRaw.acq} (n = {emg3d.shape[0]}))',
             size=fsize+4, x=.5, y=1.01)

plt.tight_layout()

plt.savefig(os.path.join(figpath, fname), dpi=300, facecolor='w',
            bbox_inches="tight",)

plt.show()

In [ ]:
from scipy import signal

In [ ]:
f_psd, psdx = signal.welch(dat3d, axis=-1, fs=fs, nperseg=fs, nfft=fs,)
psdx = np.mean(psdx, axis=0)  # takes mean per ch
psdx = np.mean(psdx, axis=0)  # takes mean over chs

try:
    plt.plot(f_psd, psdx)
except ValueError:
    for c in psdx: plt.plot(f_psd, c)
plt.xlim(0, 95)
plt.show()


In [ ]:
psdx.shape, f_psd.shape

In [ ]:
STIM = 'rest'  # 'go_left

ep_i = 5

chs_e = opm_epochs[STIM].get_data()[ep_i, :, :]
fs = recRaw.OPM_Z.info['sfreq']

for i, ch in enumerate(chs_e):
    plt.plot(np.arange(len(ch)) / fs, ch,
             label=recRaw.OPM_Z.info['ch_names'][i])

    if i > 20: break

plt.legend()

plt.close()



In [ ]:
# opm_epochs[STIM].compute_psd().plot()

# plt.show()

In [ ]:
psds, freqs = mne.time_frequency.psd_array_welch(
    opm_epochs[STIM].get_data(),
    fmin=13, fmax=20,
    n_fft=int(recRaw.OPM_Z.info['sfreq']),
    sfreq=recRaw.OPM_Z.info['sfreq'],
)



In [ ]:

psds_plot = psds.mean(axis=(0, 2))  # gives mean PSD-power within defined range per channel

mne.viz.plot_topomap(
    psds_plot,
    opm_epochs[STIM].info,
    cmap="viridis",  # for diff "RdBu_r"
    sensors=True,        # show sensor dots
    outlines="head",     # no change for meg, should add head circle, ears, nose
    contours=1,
)

plt.show()


extended cleaning


test further cleaning, HFC does not converge, try ICA for specific (stationary) artefacts

In [ ]:
recRaw.OPM_Z.get_data().shape

### Explore visualization

- calculate spectral envelops (analytical signals) for theta, alpha, beta, gamma
- plot envelops over 3 second epoch windows, average over channels and over epochs, resulting in mean envelop over the course of specific task
- compare contra-lateral vs ipsi-lateral hemisphere

- plot next to ACC-hand, and mean-envelop from EMG per extremity (deltoid + brachrad)